In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv(r"C:\Users\wasif\Downloads\ml-latest-small\ml-latest-small\ratings.csv")
print(data.shape)
data = data.drop(columns=['timestamp'])
data.head(5)

(100836, 4)


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [3]:
movie_titles = pd.read_csv(r"C:\Users\wasif\Downloads\ml-latest-small\ml-latest-small\movies.csv")
movie_titles = movie_titles.drop(columns=['genres'])
movie_titles.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [4]:
data = pd.merge(data, movie_titles, on = 'movieId')
data.head(500)

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)
...,...,...,...,...
495,369,47,3.5,Seven (a.k.a. Se7en) (1995)
496,370,47,3.5,Seven (a.k.a. Se7en) (1995)
497,372,47,5.0,Seven (a.k.a. Se7en) (1995)
498,373,47,5.0,Seven (a.k.a. Se7en) (1995)


In [5]:
moviemat = data.pivot_table(index ='userId', columns = 'title', values = 'rating')

In [6]:
def pearson_correlation(user1, user2): 
    # Extract the rows corresponding to the two users
    user1 = moviemat.loc[user1]
    user2 = moviemat.loc[user2]
    
    # Find the common movies that both users have rated (non-missing values)
    common_items = np.logical_and(~np.isnan(user1), ~np.isnan(user2))
    
    # If there are no common movies, return NaN to indicate no correlation
    if not common_items.any():
        return np.nan
    
    # Calculate the Pearson correlation for the common items
    user1_common = user1[common_items]
    user2_common = user2[common_items]

    # Calculate the means of the common ratings
    mean_user1 = np.nanmean(user1_common)
    mean_user2 = np.nanmean(user2_common)

    # Calculate the numerator and denominators for the Pearson correlation formula
    numerator = np.sum((user1_common - mean_user1) * (user2_common - mean_user2))
    denominator_user1 = np.sqrt(np.sum((user1_common - mean_user1) ** 2))
    denominator_user2 = np.sqrt(np.sum((user2_common - mean_user2) ** 2))
    if(denominator_user1*denominator_user2==0):
        return np.nan

    # Calculate the Pearson correlation coefficient
    correlation = numerator / (denominator_user1 * denominator_user2)
    
    return correlation

In [48]:
def top_similar_users(user, all_users, moviemat, num_users=10):
    # Calculate Pearson correlation between the given user and all other users
    correlations = []
    for other_user in all_users:
        if user != other_user:
            correlation = pearson_correlation(user, other_user)
            if not np.isnan(correlation):
                correlations.append((other_user, correlation))

    # Sort the correlations in descending order
    correlations.sort(key=lambda x: x[1], reverse=True)

    # Select the top num_users similar users
    top_similar_users = correlations[:num_users]

    return top_similar_users

# Example usage:
# Replace 'user' with the user you want to find similar users for and 'all_users' with the list of all users in your dataset.
# user = 1 # Replace with the actual user name or ID
# all_users = moviemat.index  # Get the list of user IDs from the index of moviemat
# top_2_similar_users = top_similar_users(user, all_users, moviemat, num_users=2)

# print("Top 2 most similar users to", user, "are:")
# for similar_user, correlation in top_2_similar_users:
#     print(similar_user, "with correlation", correlation)

Top 2 most similar users to 1 are:
146 with correlation 1.0000000000000002
550 with correlation 1.0000000000000002


In [23]:

# Extract the user IDs from the top 10 similar users list
similar_user_ids = [user_id for user_id, _ in top_2_similar_users]

# Include user=1 in the list of similar users
similar_user_ids.append(user)

# Create a new DataFrame with only the selected users
new_matrix = moviemat.loc[similar_user_ids]


In [151]:
user = 1 # Replace with the actual user name or ID
all_users = moviemat.index  # Get the list of user IDs from the index of moviemat
top_50_similar_users_1 = top_similar_users(user, all_users, moviemat, num_users=10)

print("Top 10 most similar users to", user, "are:")
for similar_user, correlation in top_50_similar_users_1:
    print(similar_user, "with correlation", correlation)

Top 50 most similar users to 1 are:
146 with correlation 1.0000000000000002
550 with correlation 1.0000000000000002
106 with correlation 1.0
333 with correlation 1.0
598 with correlation 0.9999999999999998
473 with correlation 0.9622504486493763
511 with correlation 0.9258200997725514
9 with correlation 0.9185586535436917
13 with correlation 0.8783100656536798
366 with correlation 0.8728715609439696


In [152]:
# Extract the user IDs from the top 10 similar users list
similar_user_ids_1 = [user_id for user_id, _ in top_50_similar_users_1]

# Include user=1 in the list of similar users
similar_user_ids_1.append(user)

# Create a new DataFrame with only the selected users
new_matrix_user1 = moviemat.loc[similar_user_ids_1]

In [185]:
#For 2nd user=146
user = 146 # Replace with the actual user name or ID
all_users = moviemat.index  # Get the list of user IDs from the index of moviemat
top_10_similar_users_2 = top_similar_users(user, all_users, moviemat, num_users=10)

print("Top 10 most similar users to", user, "are:")
for similar_user, correlation in top_10_similar_users_2:
    print(similar_user, "with correlation", correlation)

Top 10 most similar users to 146 are:
1 with correlation 1.0000000000000002
135 with correlation 1.0000000000000002
149 with correlation 1.0000000000000002
221 with correlation 1.0000000000000002
428 with correlation 1.0000000000000002
464 with correlation 1.0000000000000002
512 with correlation 1.0000000000000002
10 with correlation 1.0
14 with correlation 1.0
46 with correlation 1.0


In [188]:
#For 2nd user=146
user=146
# Extract the user IDs from the top 10 similar users list
similar_user_ids_2 = [user_id for user_id, _ in top_10_similar_users_2]

# Include user=1 in the list of similar users
similar_user_ids_2.append(user)

# Create a new DataFrame with only the selected users
new_matrix_user2 = moviemat.loc[similar_user_ids_2]


In [200]:
#For 3rd user=550
user = 550 # Replace with the actual user name or ID
all_users = moviemat.index  # Get the list of user IDs from the index of moviemat
top_10_similar_users_3 = top_similar_users(user, all_users, moviemat, num_users=10)

print("Top 10 most similar users to", user, "are:")
for similar_user, correlation in top_10_similar_users_3:
    print(similar_user, "with correlation", correlation)

Top 10 most similar users to 550 are:
1 with correlation 1.0000000000000002
69 with correlation 1.0000000000000002
72 with correlation 1.0000000000000002
121 with correlation 1.0000000000000002
135 with correlation 1.0000000000000002
335 with correlation 1.0000000000000002
385 with correlation 1.0000000000000002
428 with correlation 1.0000000000000002
430 with correlation 1.0000000000000002
572 with correlation 1.0000000000000002


In [201]:
#For 3rd user=550
user=550
# Extract the user IDs from the top 10 similar users list
similar_user_ids_3 = [user_id for user_id, _ in top_10_similar_users_3]

# Include user=1 in the list of similar users
similar_user_ids_3.append(user)

# Create a new DataFrame with only the selected users
new_matrix_user3 = moviemat.loc[similar_user_ids_3]


In [202]:
new_matrix_user3

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
new_matrix_user1

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
new_matrix

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN


In [25]:
new_matrix.shape

(3, 9719)

In [26]:

# Count the number of movies not rated by anyone
num_movies_not_rated_by_anyone = new_matrix.isna().all(axis=0).sum()

print(f"Number of movies not rated by anyone in group: {num_movies_not_rated_by_anyone}")



Number of movies not rated by anyone: 9432


In [33]:


# Find movies not rated by anyone
movies_not_rated_by_anyone = new_matrix.columns[new_matrix.isna().all(axis=0)]

# Print the list of movies not rated by anyone
print("Movies not rated by anyone:")
print(movies_not_rated_by_anyone)

movies_not_rated_by_anyone_list = list(movies_not_rated_by_anyone)


Movies not rated by anyone:
Index([''71 (2014)', ''Hellboy': The Seeds of Creation (2004)',
       ''Round Midnight (1986)', ''Salem's Lot (2004)',
       ''Til There Was You (1997)', ''Tis the Season for Love (2015)',
       ''burbs, The (1989)', ''night Mother (1986)',
       '(500) Days of Summer (2009)', '*batteries not included (1987)',
       ...
       'Zulu (1964)', 'Zulu (2013)', '[REC] (2007)', '[REC]² (2009)',
       '[REC]³ 3 Génesis (2012)',
       'anohana: The Flower We Saw That Day - The Movie (2013)',
       'eXistenZ (1999)', 'xXx (2002)', 'xXx: State of the Union (2005)',
       'À nous la liberté (Freedom for Us) (1931)'],
      dtype='object', name='title', length=9432)


In [252]:
# Assuming 'user' is the user for whom you want to find similar users
user = 1

# Get the list of all users
all_users = moviemat.index

# Initialize a dictionary to store similarities
similarity = {}

# Calculate Pearson correlation for each user using your custom function
for other_user in all_users:
    if user != other_user:
        correlation = pearson_correlation(user, other_user)
        
        if not np.isnan(correlation):
            similarity[other_user] = correlation
        else:
            print(f"No common movies between user {user} and user {other_user}")

# Example usage of similarity dictionary
item = 146  # Replace with the actual item you want to predict
predicted_rating = predict_rating(item, similarity, new_matrix)
print(f"Predicted rating for item {item}: {predicted_rating}")


No common movies between user 1 and user 2
No common movies between user 1 and user 12
No common movies between user 1 and user 53
No common movies between user 1 and user 60
No common movies between user 1 and user 77
No common movies between user 1 and user 85
No common movies between user 1 and user 87
No common movies between user 1 and user 127
No common movies between user 1 and user 175
No common movies between user 1 and user 180
No common movies between user 1 and user 184
No common movies between user 1 and user 190
No common movies between user 1 and user 245
No common movies between user 1 and user 251
No common movies between user 1 and user 252
No common movies between user 1 and user 253
No common movies between user 1 and user 258
No common movies between user 1 and user 259
No common movies between user 1 and user 278
No common movies between user 1 and user 281
No common movies between user 1 and user 291
No common movies between user 1 and user 306
No common movies b

In [256]:
#For user 2
# Assuming 'user' is the user for whom you want to find similar users
user = 146

# Get the list of all users
all_users = moviemat.index

# Initialize a dictionary to store similarities
similarity2 = {}

# Calculate Pearson correlation for each user using your custom function
for other_user in all_users:
    if user != other_user:
        correlation = pearson_correlation(user, other_user)
        
        if not np.isnan(correlation):
            similarity2[other_user] = correlation
        else:
            print(f"No common movies between user {user} and user {other_user}")

# Example usage of similarity dictionary
item = 146  # Replace with the actual item you want to predict
predicted_rating = predict_rating(item, similarity, new_matrix)
print(f"Predicted rating for item {item}: {predicted_rating}")



No common movies between user 146 and user 2
No common movies between user 146 and user 3
No common movies between user 146 and user 4
No common movies between user 146 and user 5
No common movies between user 146 and user 7
No common movies between user 146 and user 8
No common movies between user 146 and user 9
No common movies between user 146 and user 11
No common movies between user 146 and user 12
No common movies between user 146 and user 13
No common movies between user 146 and user 15
No common movies between user 146 and user 16
No common movies between user 146 and user 17
No common movies between user 146 and user 21
No common movies between user 146 and user 22
No common movies between user 146 and user 23
No common movies between user 146 and user 25
No common movies between user 146 and user 26
No common movies between user 146 and user 27
No common movies between user 146 and user 29
No common movies between user 146 and user 30
No common movies between user 146 and use

No common movies between user 146 and user 54
No common movies between user 146 and user 55
No common movies between user 146 and user 59
No common movies between user 146 and user 60
No common movies between user 146 and user 61
No common movies between user 146 and user 65
No common movies between user 146 and user 67
No common movies between user 146 and user 69
No common movies between user 146 and user 70
No common movies between user 146 and user 71
No common movies between user 146 and user 73
No common movies between user 146 and user 74
No common movies between user 146 and user 75
No common movies between user 146 and user 77
No common movies between user 146 and user 79
No common movies between user 146 and user 83
No common movies between user 146 and user 85
No common movies between user 146 and user 87
No common movies between user 146 and user 88
No common movies between user 146 and user 93
No common movies between user 146 and user 96
No common movies between user 146 

In [258]:
similarity

{3: 0.07981885710176258,
 4: 0.20798296652189258,
 5: 0.26874902634577136,
 6: -0.2916357806775922,
 7: -0.11877282594685312,
 8: 0.46966821831386213,
 9: 0.9185586535436917,
 10: -0.037986858819879316,
 11: -0.0876434000715537,
 13: 0.8783100656536798,
 14: 0.22908106449636373,
 15: 0.23263601431101896,
 16: 0.19185059422750425,
 17: 0.03790087381375187,
 18: 0.2053708329072329,
 19: 0.32518024337203466,
 20: 0.4196412352206233,
 21: 0.10711877914382666,
 22: -0.46303926413858626,
 23: -0.364371689814067,
 24: 0.2490463605705079,
 25: -0.16666666666666666,
 26: 0.1414213562373095,
 27: 0.18943733019219794,
 28: -0.014257366896705021,
 29: -0.0704947935554182,
 30: -0.14285714285714285,
 31: 0.03336630399750969,
 32: 0.2770173444257775,
 33: 0.08852253162596159,
 34: 0.03235585221454248,
 35: 0.2970442628930023,
 36: 0.43242556575519353,
 37: -0.4193548387096776,
 38: 0.24608488154354952,
 39: -0.22568092527182754,
 40: -0.35400521619692155,
 41: -0.1349298593551727,
 42: 0.14080922503

In [254]:
similarity2

{}

In [259]:
#For user 3
# Assuming 'user' is the user for whom you want to find similar users
user = 550

# Get the list of all users
all_users = moviemat.index

# Initialize a dictionary to store similarities
similarity3 = {}

# Calculate Pearson correlation for each user using your custom function
for other_user in all_users:
    if user != other_user:
        correlation = pearson_correlation(user, other_user)
        
        if not np.isnan(correlation):
            similarity3[other_user] = correlation
        else:
            print(f"No common movies between user {user} and user {other_user}")

# Example usage of similarity dictionary
item = 146  # Replace with the actual item you want to predict
predicted_rating = predict_rating(item, similarity, new_matrix)
print(f"Predicted rating for item {item}: {predicted_rating}")




No common movies between user 550 and user 3
No common movies between user 550 and user 4
No common movies between user 550 and user 6
No common movies between user 550 and user 7
No common movies between user 550 and user 9
No common movies between user 550 and user 12
No common movies between user 550 and user 13
No common movies between user 550 and user 20
No common movies between user 550 and user 23
No common movies between user 550 and user 26
No common movies between user 550 and user 27
No common movies between user 550 and user 29
No common movies between user 550 and user 30
No common movies between user 550 and user 31
No common movies between user 550 and user 35
No common movies between user 550 and user 36
No common movies between user 550 and user 37
No common movies between user 550 and user 39


No common movies between user 550 and user 40
No common movies between user 550 and user 44
No common movies between user 550 and user 48
No common movies between user 550 and user 51
No common movies between user 550 and user 53
No common movies between user 550 and user 55
No common movies between user 550 and user 59
No common movies between user 550 and user 61
No common movies between user 550 and user 71
No common movies between user 550 and user 74
No common movies between user 550 and user 75
No common movies between user 550 and user 79
No common movies between user 550 and user 84
No common movies between user 550 and user 85
No common movies between user 550 and user 87
No common movies between user 550 and user 89
No common movies between user 550 and user 90
No common movies between user 550 and user 92
No common movies between user 550 and user 95
No common movies between user 550 and user 97
No common movies between user 550 and user 99
No common movies between user 550 

In [260]:
similarity3

{1: 1.0000000000000002,
 2: -0.7486371059063728,
 5: -0.9999999999999998,
 8: 0.9999999999999998,
 10: -0.5428614415286325,
 11: -0.9999999999999998,
 14: -0.9999999999999998,
 15: -0.2509158102678356,
 16: -0.6454972243679026,
 17: 0.283069258536149,
 18: 0.320412176144987,
 19: 0.5000000000000001,
 21: 0.07853774405141002,
 22: -0.49884184634077,
 24: -0.1581138830084189,
 25: -0.5400617248673216,
 28: 0.6004805767690768,
 32: 0.9999999999999998,
 33: 0.30151134457776363,
 34: -0.5000000000000001,
 38: 0.9999999999999998,
 41: -0.6431758808251546,
 42: -0.5,
 43: -1.0000000000000002,
 45: 0.33333333333333337,
 46: -0.9999999999999998,
 47: 0.3086066999241839,
 49: 0.7637626158259733,
 50: -0.6729676005906329,
 52: 0.18650096164806276,
 54: 0.5000000000000001,
 56: -0.9999999999999998,
 57: 0.5,
 58: 0.9999999999999998,
 60: -0.9999999999999998,
 62: 0.20915561428987506,
 63: -0.1624536637996148,
 64: 0.5773502691896258,
 65: 0.30618621784789724,
 66: 0.30151134457776363,
 67: 0.13363

In [30]:
#New code
# Function for predicting rating of a movie
def predict_rating(item, similarity_scores, moviemat):
    # Find the users who have rated the item and have non-NaN values
    item = moviemat.columns[item]
    rated_users = ~np.isnan(moviemat[item])
    user = 1  # Specify the user for whom you want to make predictions
    
    # Get the similarity scores
    similarity = similarity_scores
    
    # Initialize the weighted sum and sum of similarity scores
    weighted_sum = 0
    total_similarity = 0
    
    # Iterate through users who have rated the item
    for other_user in moviemat.index:
        if other_user != user and rated_users[other_user]:
            # Calculate the weighted contribution to the prediction
            similarity_value = similarity.get(other_user, 0)  # Use get() to handle missing keys
            other_user_rating = moviemat.at[other_user, item]
            weighted_sum += similarity_value * (other_user_rating - np.nanmean(moviemat.loc[other_user]))
            total_similarity += np.abs(similarity_value)
    
    if total_similarity == 0:
        # Handle the case where there are no similar users
        return np.nan
    
    # Calculate the predicted rating by adding the user's mean rating
    # and the weighted sum of ratings from similar users
    user_ratings = moviemat.loc[user]
    user_mean = np.nanmean(user_ratings)
    predicted_rating = user_mean + (weighted_sum / total_similarity)
    predicted_rating = min(5, max(0, predicted_rating))
    return predicted_rating

# Example usage:
# predicted_rating = predict_rating(item, similarity, new_matrix)


In [ ]:
# # Find the movies that User 1 hasn't rated
# user = 1
# user1_ratings = moviemat.loc[user]
# unrated_movies = user1_ratings.isna()

# # Initialize a list to store predicted ratings, movie indices, and their corresponding movie titles
# predicted_ratings = []

# # Iterate through unrated movies and make predictions
# for item in range(len(new_matrix.columns)):
#     if unrated_movies[item]:
#         predicted_rating = predict_rating(item, similarity, new_matrix)
#         if not np.isnan(predicted_rating):
#             print(f"{new_matrix.columns[item]}: {predicted_rating}")

#         user_1_mat.loc[user, item] = predicted_rating
#         predicted_ratings.append((item, predicted_rating))

# # Sort the predicted ratings in descending order
# predicted_ratings.sort(key=lambda x: x[1], reverse=True)

# # Get the top 10 rated movies that User 1 hasn't rated yet
# top_10_unrated_movies = predicted_ratings[:10]

# # Extract the movie titles and ratings based on their indices
# top_10_movie_info = [(new_matrix.columns[item], rating) for item, rating in top_10_unrated_movies]

# # Print the top 10 rated movies that User 1 hasn't rated yet with their ratings
# print("Top 10 rated movies that User 1 hasn't rated yet:")
# for movie_title, rating in top_10_movie_info:
#     print(f"{movie_title}: {rating}")


In [171]:
new_matrix_user1[-1]

KeyError: -1

In [181]:
user_1_mat = new_matrix_user1.iloc[10:11].copy()
user_1_mat

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN


In [190]:
#For user 2

user_2_mat = new_matrix_user2.iloc[10:11].copy()
user_2_mat

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [203]:
#For user 3

user_3_mat = new_matrix_user3.iloc[10:11].copy()
user_3_mat

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [261]:
# New code
# Find the movies that User 1 hasn't rated
user = 1
user1_ratings = new_matrix_user1.loc[user]
unrated_movies = user1_ratings.isna()

# Initialize a list to store predicted ratings, movie indices, and their corresponding movie titles
predicted_ratings = []

# Iterate through unrated movies and make predictions
for item in range(len(new_matrix_user1.columns)):
    if unrated_movies[item]:
        predicted_rating = predict_rating(item, similarity, moviemat)
        # if not np.isnan(predicted_rating):
        #    print(f"{new_matrix_user1.columns[item]}: {predicted_rating}")

        # Update user_1_mat directly with the predicted rating
        column_name = new_matrix_user1.columns[item]
        user_1_mat.loc[user, column_name] = predicted_rating
        predicted_ratings.append((column_name, predicted_rating))

# Sort the predicted ratings in descending order
predicted_ratings.sort(key=lambda x: x[1], reverse=True)

# Get the top 10 rated movies that User 1 hasn't rated yet
#top_10_unrated_movies = predicted_ratings[:10]

# Extract the movie titles and ratings based on their indices
#top_10_movie_info = [(new_matrix.columns[item], rating) for item, rating in top_10_unrated_movies]

# Print the top 10 rated movies that User 1 hasn't rated yet with their ratings
print("Top 10 rated movies that User 1 hasn't rated yet:")
for movie_title, rating in top_10_movie_info:
    print(f"{movie_title}: {rating}")


C:\Users\wasif\AppData\Local\Temp\ipykernel_11052\2939830986.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if unrated_movies[item]:


Top 10 rated movies that User 1 hasn't rated yet:
'71 (2014): nan
'Hellboy': The Seeds of Creation (2004): nan
'Round Midnight (1986): nan
'Salem's Lot (2004): nan
'Til There Was You (1997): nan
'Tis the Season for Love (2015): nan
'burbs, The (1989): nan
'night Mother (1986): nan
(500) Days of Summer (2009): nan
*batteries not included (1987): nan


In [262]:
#For User 2
# New code
# Find the movies that User 1 hasn't rated
user = 146
user2_ratings = new_matrix_user2.loc[user]
unrated_movies = user2_ratings.isna()

# Initialize a list to store predicted ratings, movie indices, and their corresponding movie titles
predicted_ratings = []

# Iterate through unrated movies and make predictions
for item in range(len(new_matrix_user2.columns)):
    if unrated_movies[item]:
        predicted_rating = predict_rating(item, similarity2, moviemat)
        # if not np.isnan(predicted_rating):
        #    print(f"{new_matrix_user1.columns[item]}: {predicted_rating}")

        # Update user_1_mat directly with the predicted rating
        column_name = new_matrix_user2.columns[item]
        user_2_mat.loc[user, column_name] = predicted_rating
        predicted_ratings.append((column_name, predicted_rating))

# Sort the predicted ratings in descending order
#predicted_ratings.sort(key=lambda x: x[1], reverse=True)

# Get the top 10 rated movies that User 1 hasn't rated yet
#top_10_unrated_movies = predicted_ratings[:10]

# Extract the movie titles and ratings based on their indices
#top_10_movie_info = [(new_matrix.columns[item], rating) for item, rating in top_10_unrated_movies]

# Print the top 10 rated movies that User 1 hasn't rated yet with their ratings
# print("Top 10 rated movies that User 1 hasn't rated yet:")
# for movie_title, rating in top_10_movie_info:
#     print(f"{movie_title}: {rating}")


C:\Users\wasif\AppData\Local\Temp\ipykernel_11052\565422314.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if unrated_movies[item]:


In [263]:
#For User 3
# New code
# Find the movies that User 1 hasn't rated
user = 550
user3_ratings = new_matrix_user3.loc[user]
unrated_movies = user3_ratings.isna()

# Initialize a list to store predicted ratings, movie indices, and their corresponding movie titles
predicted_ratings = []

# Iterate through unrated movies and make predictions
for item in range(len(new_matrix_user3.columns)):
    if unrated_movies[item]:
        predicted_rating = predict_rating(item, similarity3, moviemat)
        # if not np.isnan(predicted_rating):
        #    print(f"{new_matrix_user1.columns[item]}: {predicted_rating}")

        # Update user_1_mat directly with the predicted rating
        column_name = new_matrix_user3.columns[item]
        user_3_mat.loc[user, column_name] = predicted_rating
        predicted_ratings.append((column_name, predicted_rating))

# Sort the predicted ratings in descending order
#predicted_ratings.sort(key=lambda x: x[1], reverse=True)

# Get the top 10 rated movies that User 1 hasn't rated yet
#top_10_unrated_movies = predicted_ratings[:10]

# Extract the movie titles and ratings based on their indices
#top_10_movie_info = [(new_matrix.columns[item], rating) for item, rating in top_10_unrated_movies]

# Print the top 10 rated movies that User 1 hasn't rated yet with their ratings
# print("Top 10 rated movies that User 1 hasn't rated yet:")
# for movie_title, rating in top_10_movie_info:
#     print(f"{movie_title}: {rating}")


C:\Users\wasif\AppData\Local\Temp\ipykernel_11052\472173497.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if unrated_movies[item]:


In [265]:
user_3_mat


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
550,4.054935,3.96563,4.46563,NaN,NaN,2.605657,3.988502,3.967423,4.61699,4.496454,...,3.019008,4.461231,4.802226,4.566842,3.661155,4.659989,4.145344,3.483887,3.726868,NaN


In [156]:
user_1_mat = moviemat.head(1).copy()
user_1_mat


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN


In [106]:
new_var = moviemat.iloc[1:2].copy()
new_var


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [266]:
user_1_mat


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.054935,4.767128,4.558392,3.269605,3.574265,2.605657,4.310777,3.967423,4.661113,4.449123,...,3.019008,4.861062,4.556629,4.230219,NaN,4.636781,3.442791,3.720231,4.0,5.0


In [178]:
non_nan_count = user_1_mat.count().sum()
print("Total non-NaN values in user_1_mat:", non_nan_count)


Total non-NaN values in user_1_mat: 18501


In [133]:
user_1_mat = user_1_mat.dropna(axis=1, how='all')


In [243]:
non_nan_count = user_3_mat.count().sum()
print("Total non-NaN values in user_1_mat:", non_nan_count)

Total non-NaN values in user_1_mat: 28


In [236]:
user_1_mat

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.054935,4.767128,4.558392,3.269605,3.574265,2.605657,4.310777,3.967423,4.661113,4.449123,...,3.019008,4.861062,4.556629,4.230219,NaN,4.636781,3.442791,3.720231,4.0,5.0


In [267]:

user_2_mat

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
146,4.677823,4.767128,4.016885,5.0,5.0,NaN,3.785561,3.967423,4.69707,4.5086,...,3.019008,4.821116,4.041756,3.682124,3.661155,4.747605,3.669275,2.764814,3.851098,NaN


In [302]:
user_1_mat["Stone Reader (2002)"]

userId
1    4.972698
Name: Stone Reader (2002), dtype: float64

In [270]:
merged_df = pd.concat([user_1_mat, user_2_mat, user_3_mat], axis=0)
merged_df

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.054935,4.767128,4.558392,3.269605,3.574265,2.605657,4.310777,3.967423,4.661113,4.449123,...,3.019008,4.861062,4.556629,4.230219,NaN,4.636781,3.442791,3.720231,4.000000,5.0
146,4.677823,4.767128,4.016885,5.000000,5.000000,NaN,3.785561,3.967423,4.697070,4.508600,...,3.019008,4.821116,4.041756,3.682124,3.661155,4.747605,3.669275,2.764814,3.851098,NaN
550,4.054935,3.965630,4.465630,NaN,NaN,2.605657,3.988502,3.967423,4.616990,4.496454,...,3.019008,4.461231,4.802226,4.566842,3.661155,4.659989,4.145344,3.483887,3.726868,NaN


In [272]:
#merged_df
merged_df_new = merged_df.dropna(axis=1)
merged_df_new


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),10 (1979),...,Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.054935,4.767128,4.558392,4.310777,3.967423,4.661113,4.449123,4.388685,5.0,4.480970,...,4.365022,5.000000,3.019008,4.861062,4.556629,4.230219,4.636781,3.442791,3.720231,4.000000
146,4.677823,4.767128,4.016885,3.785561,3.967423,4.697070,4.508600,4.182624,5.0,3.889919,...,4.624343,4.163011,3.019008,4.821116,4.041756,3.682124,4.747605,3.669275,2.764814,3.851098
550,4.054935,3.965630,4.465630,3.988502,3.967423,4.616990,4.496454,3.625136,5.0,4.499789,...,4.392819,4.036652,3.019008,4.461231,4.802226,4.566842,4.659989,4.145344,3.483887,3.726868


In [296]:
# Keep columns with at least one unique value (excluding NaN)
merged_df_new = merged_df.loc[:, merged_df.nunique() > 2]

merged_df_new


title,'Round Midnight (1986),"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),...,Zoolander 2 (2016),Zootopia (2016),Zulu (1964),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.558392,4.310777,4.661113,4.449123,4.388685,4.480970,3.065864,4.281805,4.547473,4.085034,...,3.952651,4.365022,5.000000,4.861062,4.556629,4.230219,4.636781,3.442791,3.720231,4.000000
146,4.016885,3.785561,4.697070,4.508600,4.182624,3.889919,3.519008,4.297842,4.486903,4.453645,...,3.519290,4.624343,4.163011,4.821116,4.041756,3.682124,4.747605,3.669275,2.764814,3.851098
550,4.465630,3.988502,4.616990,4.496454,3.625136,4.499789,2.764754,4.436224,4.476330,4.132017,...,3.575976,4.392819,4.036652,4.461231,4.802226,4.566842,4.659989,4.145344,3.483887,3.726868


In [304]:
# Calculate the mean rating for each movie
mean_ratings = merged_df_new.mean()

# Sort the movies by mean rating in descending order
sorted_mean_ratings = mean_ratings.sort_values(ascending=False)

# Print the top 10 highest-rated movies
top_10_movies = sorted_mean_ratings.head(10)

# Print the results
print("Top 10 highest-rated movies following Average Method:")
for title, rating in top_10_movies.items():
    print(f"{title}: {rating}")


Top 10 highest-rated movies following Average Method:
Lagaan: Once Upon a Time in India (2001): 4.986887339692196
Warriors, The (1979): 4.9843749831548045
Widow of St. Pierre, The (Veuve de Saint-Pierre, La) (2000): 4.983148763631385
Gold Rush, The (1925): 4.981957343267612
Promises (2001): 4.979076133181834
84 Charing Cross Road (1987): 4.977938671175509
Kings of Summer, The (2013): 4.977644624966058
Stone Reader (2002): 4.970739165431223
Nicholas Nickleby (2002): 4.970739165431223
Paradise Now (2005): 4.970739165431223


In [306]:
# Calculate the minimum rating given by each user for each movie
min_ratings_by_user = merged_df_new.min()

# Sort the movies by the minimum rating in descending order (highest ratings among the lowest)
sorted_min_ratings_least = min_ratings_by_user.sort_values(ascending=False)


# Print the top 10 movies with the highest ratings among the lowest ratings given by users
top_10_least_misery = sorted_min_ratings_least.head(10)

# Print the results
print("Top 10 movies following Least Misery Method: ")
for title, rating in top_10_least_misery.items():
    print(f"{title}: {rating}")


Top 10 movies following Least Misery Method: 
Stone Reader (2002): 4.968936959751475
Nicholas Nickleby (2002): 4.968936959751475
Paradise Now (2005): 4.968936959751475
Promises (2001): 4.967422953608585
Brief Encounter (1946): 4.967422953608585
Warriors, The (1979): 4.966928454865915
84 Charing Cross Road (1987): 4.966393059917943
Lagaan: Once Upon a Time in India (2001): 4.963048587694769
Gold Rush, The (1925): 4.958258948153646
Kings of Summer, The (2013): 4.952161568638975


In [309]:
# Calculate the mean rating for each movie
mean_ratings = merged_df_new.mean()

# Extract the data for the top 10 highest-rated movies
top_10_movies_data = merged_df_new[top_10_movies.index]

# Initialize a dictionary to store standard deviation for each user
std_deviation_by_user = {}

# Initialize a variable to store the sum of standard deviations
total_std_deviation = 0

# Calculate the standard deviation for each user's ratings for the top 10 movies
for user in merged_df_new.index:
    user_ratings = top_10_movies_data.loc[user].dropna()
    mean_user_rating = user_ratings.mean()
    squared_diff = sum((rating - mean_user_rating)**2 for rating in user_ratings)
    std_dev_user = (squared_diff / len(user_ratings))**0.5
    std_deviation_by_user[user] = std_dev_user
    total_std_deviation += std_dev_user

# Print the standard deviation for each user
print("Standard Deviation for Each User in Average Method:")
for user, std_dev in std_deviation_by_user.items():
    print(f"User {user}: {std_dev}")

# Print the sum of standard deviations
print(f"\nSum of Standard Deviations for All Users in Average Method: {total_std_deviation}")


Standard Deviation for Each User:
User 1: 0.013103047810463132
User 146: 0.013542946128135077
User 550: 0.01394885252920689

Sum of Standard Deviations for All Users: 0.0405948464678051


In [310]:
# Extract the data for the top 10 movies following Least Misery Method
top_10_least_misery_data = merged_df_new[top_10_least_misery.index]

# Initialize a dictionary to store standard deviation for each user (Least Misery Method)
std_deviation_by_user_least_misery = {}

# Initialize a variable to store the sum of standard deviations (Least Misery Method)
total_std_deviation_least_misery = 0

# Calculate the standard deviation for each user's ratings for the top 10 movies (Least Misery Method)
for user in merged_df_new.index:
    user_ratings_least_misery = top_10_least_misery_data.loc[user].dropna()
    mean_user_rating_least_misery = user_ratings_least_misery.mean()
    squared_diff_least_misery = sum((rating - mean_user_rating_least_misery)**2 for rating in user_ratings_least_misery)
    std_dev_user_least_misery = (squared_diff_least_misery / len(user_ratings_least_misery))**0.5
    std_deviation_by_user_least_misery[user] = std_dev_user_least_misery
    total_std_deviation_least_misery += std_dev_user_least_misery

# Print the standard deviation for each user (Least Misery Method)
print("Standard Deviation for Each User (Least Misery Method):")
for user, std_dev_least_misery in std_deviation_by_user_least_misery.items():
    print(f"User {user}: {std_dev_least_misery}")

# Print the sum of standard deviations (Least Misery Method)
print(f"\nSum of Standard Deviations for All Users (Least Misery Method): {total_std_deviation_least_misery}")


Standard Deviation for Each User (Least Misery Method):
User 1: 0.012001521256959635
User 146: 0.013147689300638163
User 550: 0.013417862818615664

Sum of Standard Deviations for All Users (Least Misery Method): 0.038567073376213463


In [ ]:
#observation e lekha lagbe je Least Misery te standard deviation beshi mane ei situation e ekhane manush er majhe satisfaction er difference beshi